In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
links = {}

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    
    for pair in pairs:
        links[pair] = pairs[pair]['wiki']

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian',
 'English-French': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French'}

In [3]:
base_link = "https://en.m.wiktionary.org"


In [8]:
def get_words_ipas(dest_lang, src_lang, link, page_num, lst_invalid_words=['plural', 'not comparable']):
    borrowed_words = []

    try:
        # get list of words
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all("div", class_="mw-category-group")

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                src_word = None

                for b_word in list_words:
                    borrowed_word = b_word.text
                    
                    
                    # for each loaned word get the wiki link and the source word in src lang for it.
                    ipa_page = requests.get(base_link + b_word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')
                    
                    src_word_html_final = None
                    source_word = ''
                    for src_word_html in ipa_soup.find_all("span", class_="etyl"):
                        if src_word_html != None and src_word_html.find(lambda tag: tag.name == 'a' and src_lang.lower() in tag.text.lower()):
                            
                            src_word_html_final = src_word_html.find_next("i")
                            src_word = src_word_html_final.text if src_word_html_final != None else ''
                            if src_word not in lst_invalid_words:
                                source_word = src_word
                            else:
                                src_word_html_final = src_word_html.find_next("strong")
                                src_word = src_word_html_final.text if src_word_html_final != None else ''
                                source_word = src_word
                            
                            
                            break

                    
                    # save the result so far.
                    # print([borrowed_word, source_word])
                    borrowed_words.append([borrowed_word, source_word])

        print(dest_lang, src_lang, page_num, "done", len(borrowed_words))
        
        # find whet her there is a next page containing more loan words if there is scrape that as well. 
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )

        # return borrowed_words
        if next_link != None:
            more_words = get_words_ipas(dest_lang, src_lang, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words

    except Exception as e:
        # errors can be due to timeouts/connections refused due to rate limiting. can be fixed via proxies/vpns/timeouts
        print(pair, page_num, "got error", e, len(borrowed_words))
        return borrowed_words
        
    return borrowed_words

In [9]:
for pair in links:
    if os.path.exists("results/{}.csv".format(pair)):
        overwrite = input("{}.csv exists. Overwrite existing file? (y/n) ".format(pair))
    if overwrite == "y" or not os.path.exists("results/{}.csv".format(pair)):
        print()
        [dest, src] = pair.split('-')
        words = get_words_ipas(dest, src, links[pair], 1)
        df = pd.DataFrame(words, columns=['loan_word', 'original_word'])
        df.to_csv("results/{}.csv".format(pair), index=False)
        print(pair, "done\n")


English French 1 done 200
English French 2 done 200
English French 3 done 200
English French 4 done 200
English French 5 done 200
English French 6 done 200
English-French 7 got error HTTPSConnectionPool(host='en.m.wiktionary.org', port=443): Max retries exceeded with url: /wiki/clinician (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14fcb4a00>: Failed to establish a new connection: [Errno 60] Operation timed out')) 135
English-French done

